In [1]:
import os
vipsbin = r"c:\vips-dev-8.15\bin"
os.environ['PATH'] = vipsbin + ';' + os.environ['PATH']

In [2]:
import pyvips
from tqdm.notebook import tqdm_notebook
import time
import inspect
from tifffile import TiffFile
from PIL import Image
import os
from xml.etree import ElementTree

In [3]:
def process_qptiff_image(img_path, folder, desired_file_size):
    file_name = os.path.splitext(os.path.basename(img_path))[0]
    with TiffFile(img_path) as tif:
        # Detects original size on the piramidal image
        max_height, max_width = get_max_size(img_path)
        if is_multiplex(img_path):
            channels = get_channel_names(img_path)
        counter = 0
        for i, page in enumerate(tif.pages):
            pages = str(tif.pages[i])
            # Get the resolution for each image
            height, width = get_resolution(pages)
            page_dict = create_page_dict(pages, max_height, max_width, height, width)
            
            if str(page_dict["downsampling_factor"]) == desired_file_size:
                create_folder(os.path.join(folder,"processed"+ "_" +desired_file_size))
                array = page.asarray()
                image = pyvips.Image.new_from_array(array)
                if is_multiplex(img_path):
                    image.write_to_file(os.path.join(folder,
                                                 "processed"+ "_" +desired_file_size,
                                                 file_name +"_"+channels[counter]+".tif")
                                    , pyramid=True
                                    , tile=True)
                    counter += 1
                else:
                    image.write_to_file(os.path.join(folder,
                                                 "processed"+ "_" +desired_file_size,
                                                 file_name+".tif")
                                    , pyramid=True
                                    , tile=True)

def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        #print(f"Folder '{folder_path}' created successfully.")
    else:
        pass
        #print(f"Folder '{folder_path}' already exists.")

def get_qptiff_paths(path):
    path_list = []
    for paths in os.walk(path, topdown=False):
        root = paths[0]
        for file in paths[-1]:
            if file.split(".")[-1] == "qptiff":
                path_list.append(os.path.join(root, file))

    return path_list

def get_resolution(input_str):
    input_str = str(input_str) # Make sure is str format
    size = input_str.split(" ")[4]
    height, width = size.split("x")[0], size.split("x")[1]
    return int(height), int(width)

def get_max_size(path):
    with TiffFile(path) as tif:
        height, width = -1, -1
        for i, page in enumerate(tif.pages):
            pages = str(tif.pages[i])
            img_height, img_width = get_resolution(pages)
            height = img_height if height < img_height else height
            width = img_width if width < img_width else width
    
    return height, width


def create_page_dict(page, max_height, max_width, height, width):
    is_rgb = True if "rgb" in page else False
    is_reduced = True if "reduced" in page else False
    downsampling_factor = "original" if int(max_height/height) == 1 else int(max_height/height)
    page_split = str(page).split(" ")
    result = {
        "page_number":page_split[1],
        "memory_position":page_split[2],
        "image_size": page_split[4],
        "is_rgb": is_rgb,
        "is_reduced": is_reduced,
        "downsampling_factor": downsampling_factor
    }    
    return result
    

def is_multiplex(path):
    with TiffFile(path) as tif:
        if len(tif.pages) < 30:
            return False
    return True

def check_qptiff_data(path):
    with TiffFile('./data/first_multiplex/set1/LM015_02_Set1_023_Scan1.qptiff') as tif:
        for i, page in enumerate(tif.pages):
            print(page)

def get_channel_names(img_path):
    channels = []
    with TiffFile(img_path) as tif:
        for page in tif.series[0].pages:
            channel = ElementTree.fromstring(page.description).find('Name').text
            channel = channel.replace(" ", "_")
            channels.append(channel)
    return channels
            
    

# Check multiplex Fluorescence format

The multiplex images contains multiple channels as they display images in piramidal format.
- The first 8 images will have the actual channels full resolution.
- The next 8 sets will contain piramidal versions of the images that are downsampled versions up to x64.

In [4]:
#img_path = "./data/first_multiplex/set1/LM015_02_Set1_023_Scan1.qptiff"
#check_qptiff_data(img_path)
#img_path = "./data/66-4/H21-066.4_HE332_001/Scan1/H21-066.4_HE332_001_Scan1.qptiff"
#is_multiplex(img_path)

In [5]:
#folder =  "./data/first_multiplex/set1"
folder = "./data/66-4/H21-066.4_HE332_033"

#Available resolutions Original, *2, *4, *8, *16, *32, *64

def resize_qptiff_files(folder, target_size): 
    files = get_qptiff_paths(folder)
    for file in tqdm_notebook(files):
        process_qptiff_image(file, folder, target_size)
        
        
resize_qptiff_files(folder, "original")

  0%|          | 0/1 [00:00<?, ?it/s]

## PLAYING WITH SCALING ...

In [14]:
thumb = pyvips.Image.thumbnail("./tryout/your_file0.tiff", 2048)
thumb.write_to_file("./tryout/thumbnail0.tiff")

In [ ]:
#You can use the more general affine function. An affine transformation does scaling, rotation and translation all at once. 
# You can parameterize the affine matrix to do 0 rotation and 0 translation.
#You first calculate the necessary factors to achieve the target shape

height_factor = target_height / img.height
width_factor = target_width / img.width

img.affine((width_factor, 0, 0, height_factor))

In [53]:
from xml.etree import ElementTree

new_path =  "./data/66-1/H21-066.1_HE331_0001_Scan1.qptiff"
new_path = "./data/first_multiplex/set1/LM015_02_Set1_023_Scan1.qptiff"


print(get_channel_names(new_path))

with TiffFile(new_path) as tif:
    for i, page in enumerate(tif.pages):
        pages = str(tif.pages[i])
        #print(pages)
    

['DAPI', 'Opal 570', 'Opal 690', 'Opal 480', 'Opal 620', 'Opal 780', 'Opal 520', 'Sample AF']


## OBSOLETE CODE

In [ ]:
# This function is obsolete
# iterates a folder and its subfolders gets the qptiff files and convert them to a predefined size and add them in a processed_images folder
def resize_qptiff_files(path, target_size):
    create_folder(os.path.join(path, 'processed_images'))
    for i, file in tqdm_notebook(enumerate(get_qptiff_paths(path))):
        output = 'test' + str(i) + '.tif'
        print(i, file)
        thumb = pyvips.Image.thumbnail(file, target_size)
        # Insert image path and desired size of the resulting image (need to do more research if different than squared is possible)
        print(f"Saving {output} into {os.path.join(path, 'processed_images', output)}")
        thumb.write_to_file(os.path.join(path, 'processed_images', output))

# resize_qptiff_files("./data/first_multiplex/set1/", 4096)
        